In [61]:
#pip install google-cloud-bigquery これ必要
#pip install pydata_google_auth
#pip install db-dtypes

In [167]:
#インポート文
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRanker

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

from sklearn.metrics import confusion_matrix

import optuna

import pydata_google_auth
from google.cloud import bigquery 

import pickle

In [168]:
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [169]:
client = bigquery.Client(project='keiba-381006', credentials=credentials)

In [170]:
race_query = '''
    SELECT * 
    FROM osaka.race_table
    '''
race_df = client.query(race_query).to_dataframe()

In [171]:
pd.set_option('display.max_columns', 100)
race_df.head(1)

,race_id,race_round,race_title,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,is_obstacle,ground_type,is_left_right_straight,distance,datetime
0,201709020411,11,第61回大阪杯(G1),0,1,2017-04-02,9,14,4,5,3,4,8,13,240,110,280,280,2130,2320,540,480,1800,3390,4830,23910,0,1,1,2000,2017-04-02 15:40:00+00:00


In [172]:
horse_query = '''
    SELECT * 
    FROM osaka.horse_table 
    '''
horse_df = client.query(horse_query).to_dataframe()

In [173]:
pd.set_option('display.max_columns', 100)
horse_df.head(1)

,race_id,rank,frame_number,horse_number,horse_id,burden_weight,rider_id,goal_time,goal_time_dif,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,is_down,age,sex,horse_weight_dif,burden_weight_rate,avg_velocity
0,202009020411,2,8,12,2016104750,55,1102,118.4,0.0,3.0,34,5.2,4,454,1151,226800,0,4,1,-6,0.121145,16.891892


In [174]:
horse_info_query = '''
    SELECT * 
    FROM osaka.horse_info_table
    '''
horse_info_df = client.query(horse_info_query).to_dataframe()

In [175]:
pd.set_option('display.max_columns', 100)
horse_info_df.head(1)

,horse_id,bday,tame_id,owner_id,producer_id,production_area,auction_price,winnings,lifetime_record,wined_race_title,inbreeding_1,inbreeding_2,father,faths_father,faths_mother,mother,moths_father,moths_mother
0,2012104463,2012年3月17日,01128,471033,393126,千歳市,928万円 (2014年 千葉サラブレッド・セール),"2億3,061万円 (中央)",59戦9勝 [9-8-5-37],201803020811,2011104480,0,2003102205,000a000d77,1997102385,2004103210,1998101516,000a00fa62


In [176]:
horse_race_query = '''
    SELECT * 
    FROM osaka.horse_race_table
    '''
horse_race_df = client.query(horse_race_query).to_dataframe()

In [177]:
pd.set_option('display.max_columns', 100)
horse_race_df.head(1)

,date,where_racecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,popular,rank,rider_id,burden_weight,distance,ground_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner_up_horse_id,prize,horse_id,target_race_id,race_rank,horse_weight_dif,ground_type,race_date
0,2014-11-01,8,1,9,萩S(OP),201408040809,6,4,4,5.8,3,5,1018,55.0,1800,1,1:48.0,0.4,6-6,35.9-34.8,34.3,448,2012105703,150.0,2012104463,201809020411,0,0,1,2018-04-01


In [178]:
targetrace_query = '''
    SELECT * 
    FROM osaka.targetrace_table
    '''
targetrace_df = client.query(targetrace_query).to_dataframe()

In [179]:
pd.set_option('display.max_columns', 100)
targetrace_df.head(1)

,race_id,race_title,date,race_round,weather,ground_status,where_racecourse,total_horse_number,race_rank,is_obstacle,ground_type,is_left_right_straight,distance,datetime
0,202309020411,大阪杯(G1),2023-04-02,11,0,1,9,16,3,0,1,1,2000,2023-04-02 15:40:00+00:00


In [180]:
targethorse_query = '''
    SELECT * 
    FROM osaka.targethorse_table
    '''
targethorse_df = client.query(targethorse_query).to_dataframe()

In [181]:
pd.set_option('display.max_columns', 100)
targethorse_df.head(1)

,race_id,frame_number,horse_number,horse_id,burden_weight,rider_id,tamer_id,horse_weight,odds,popular,horse_name,age,sex,horse_weight_dif,burden_weight_rate
0,202309020411,1,1,2018105081,56,1174,1151,472,None,None,ジェラルディーナ,5,1,2,0.118644


In [182]:
targethorse_info_query = '''
    SELECT * 
    FROM osaka.targethorse_info_table
    '''
targethorse_info_df = client.query(targethorse_info_query).to_dataframe()

In [183]:
pd.set_option('display.max_columns', 100)
targethorse_info_df.head(1)

,horse_id,bday,tame_id,owner_id,producer_id,production_area,lifetime_record,wined_race_title,inbreeding_1,inbreeding_2,father,faths_father,faths_mother,mother,moths_father,moths_mother
0,2018100927,2018年4月6日,1101,708800,803320,日高町,18戦5勝 [5-4-3-6],202210040211,2019102071,2.020102e+09,1998110135,000a00013a,000a000071,2012104864,2002100816,2006103348


In [184]:
targethorse_race_query = '''
    SELECT * 
    FROM osaka.targethorse_race_table
    '''
targethorse_race_df = client.query(targethorse_race_query).to_dataframe()

In [185]:
pd.set_option('display.max_columns', 100)
targethorse_race_df.head(1)

,date,where_racecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,popular,rank,rider_id,burden_weight,distance,ground_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner_up_horse_id,horse_id,race_rank,horse_weight_dif,ground_type
0,2019-10-26,8,0,9,萩S(L),201908040809,7,4,4,2.2,1,1,5271,55.0,1800,3,1:50.8,-0.1,3-3,37.5-34.7,34.5,490,2.017105e+09,2017105567,0,0,1


In [186]:
horse_columns = ['rank',
                         'race_id',
                          'horse_id',
                          'rider_id',
                          'horse_number',
                          'age',
                          'sex',
#                           'odds',
#                           'popular',
                          'horse_weight'
                         ]
targethorse_columns = [
                         'race_id',
                          'horse_id',
                          'rider_id',
                          'horse_number',
                          'age',
                          'sex',
#                           'odds',
#                           'popular',
                          'horse_weight'
                         ]
df_main = horse_df[horse_columns]
df_target = targethorse_df[targethorse_columns]

In [187]:
#データのマージ
df_tmp_1 = race_df[['race_id',
                      'ground_status',
                     ]]
df_main_1 = pd.merge(df_main, df_tmp_1, on='race_id', how='left')

df_tmp_1 = targetrace_df[['race_id',
                      'ground_status',
                     ]]
df_target_1 = pd.merge(df_target, df_tmp_1, on='race_id', how='left')

In [188]:
# データのマージ
df_tmp_2 = horse_info_df[['horse_id',
                       'tame_id',
                       'inbreeding_1',
                       'inbreeding_2'
                     ]]
#重複データの削除
df_tmp_2 = df_tmp_2.drop_duplicates(subset=['horse_id'])

df_main_2 = pd.merge(df_main_1, df_tmp_2, on='horse_id', how='left')

df_tmp_2 = targethorse_info_df[['horse_id',
                       'tame_id',
                       'inbreeding_1',
                       'inbreeding_2'
                     ]]

df_target_2 = pd.merge(df_target_1, df_tmp_2, on='horse_id', how='left')

In [189]:
df_main_2

,rank,race_id,horse_id,rider_id,horse_number,age,sex,horse_weight,ground_status,tame_id,inbreeding_1,inbreeding_2
0,2,202009020411,2016104750,1102,12,4,1,454,1,01151,202006050811,0
1,1,202109020411,2017105335,1088,8,4,1,422,4,01117,202109020411,0
2,8,202209020411,2018105554,1014,11,4,1,498,1,01151,202109020609,0
3,4,201709020411,2013105788,5339,14,4,0,502,1,01061,2010104042,2016104415
4,7,201709020411,2013106099,1018,1,4,0,480,1,01002,2015105013,2011104074
...,...,...,...,...,...,...,...,...,...,...,...,...
80,7,202209020411,2015101654,1124,15,7,2,468,1,01028,2017106602,2010100697
81,9,201809020411,2010102459,700,6,8,1,474,1,01058,2014104273,2007100100
82,10,201709020411,2009102678,700,9,8,0,500,1,01013,2007105728,2008105790
83,15,201809020411,2010103602,732,10,8,0,510,1,01071,2014102878,2004102759


In [190]:
df_target_2

,race_id,horse_id,rider_id,horse_number,age,sex,horse_weight,ground_status,tame_id,inbreeding_1,inbreeding_2
0,202309020411,2018105081,1174,1,5,1,472,1,1151,202209050411,NaN
1,202309020411,2018100927,1115,2,5,1,428,1,1101,2019102071,2.020102e+09
2,202309020411,2019104740,5339,11,4,1,484,1,1118,202205021011,NaN
3,202309020411,2016100915,1171,3,7,0,498,1,1142,2014100870,2.020106e+09
4,202309020411,2018102348,5203,4,5,0,488,1,1145,2020105998,2.017102e+09
5,202309020411,2018105074,1018,5,5,0,472,1,1129,2019105240,2.020103e+09
6,202309020411,2017105567,1088,6,6,0,498,1,1071,202307010611,NaN
7,202309020411,2019102879,660,7,4,0,476,1,1044,2009101399,2.015102e+09
8,202309020411,2019101782,5386,8,4,0,494,1,420,2020101712,NaN
9,202309020411,2018100274,666,9,5,0,512,1,1062,2019101507,2.014102e+09


##### ラベルエンコーディング

In [191]:
df_target_2['horse_weight'] = df_target_2['horse_weight'].astype(float)

In [192]:
df_target_2['rank']=0

In [193]:
df_merged = pd.concat([df_target_2, df_main_2], axis=0)

In [194]:
le = LabelEncoder()
df_merged = pd.concat([df_target_2, df_main_2], axis=0)

In [197]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 84
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   race_id        101 non-null    Int64  
 1   horse_id       101 non-null    Int64  
 2   rider_id       101 non-null    Int64  
 3   horse_number   101 non-null    Int64  
 4   age            101 non-null    Int64  
 5   sex            101 non-null    Int64  
 6   horse_weight   101 non-null    Float64
 7   ground_status  101 non-null    Int64  
 8   tame_id        101 non-null    object 
 9   inbreeding_1   101 non-null    object 
 10  inbreeding_2   95 non-null     Float64
 11  rank           101 non-null    Int64  
dtypes: Float64(2), Int64(8), object(2)
memory usage: 11.2+ KB


In [195]:
tame_id = le.fit(df_merged['tame_id'])
df_main_2['tame_id'] = le.transform(df_main_2['tame_id'])
df_target_2['tame_id'] = le.transform(df_target_2['tame_id'])

inbreeding_1 = le.fit(df_merged['inbreeding_1'])
df_main_2['inbreeding_1'] = le.transform(df_main_2['inbreeding_1'])
df_target_2['inbreeding_1'] = le.transform(df_target_2['inbreeding_1'])

inbreeding_2 = le.fit(df_merged['inbreeding_2'])
df_main_2['inbreeding_2'] = le.transform(df_main_2['inbreeding_2'])
df_target_2['inbreeding_2'] = le.transform(df_target_2['inbreeding_2'])

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str']

In [196]:
df_main_2['rank'] = df_main_2['rank'].astype('int64')
df_main_2['race_id'] = df_main_2['race_id'].astype('int64')
df_main_2['horse_id'] = df_main_2['horse_id'].astype('int64')
df_main_2['rider_id'] = df_main_2['rider_id'].astype('int64')
df_main_2['horse_number'] = df_main_2['horse_number'].astype('int64')
df_main_2['age'] = df_main_2['age'].astype('int64')
df_main_2['sex'] = df_main_2['sex'].astype('int64')
# df_main_2['popular'] = df_main_2['popular'].astype('int64')
df_main_2['ground_status'] = df_main_2['ground_status'].astype('int64')

In [117]:
df_target_2['rank'] = df_target_2['rank'].astype('int64')
df_target_2['race_id'] = df_target_2['race_id'].astype('int64')
df_target_2['horse_id'] = df_target_2['horse_id'].astype('int64')
df_target_2['rider_id'] = df_target_2['rider_id'].astype('int64')
df_target_2['horse_number'] = df_target_2['horse_number'].astype('int64')
df_target_2['age'] = df_target_2['age'].astype('int64')
df_target_2['sex'] = df_target_2['sex'].astype('int64')
# df_target_2['popular'] = df_target_2['popular'].astype('int64')
df_target_2['ground_status'] = df_target_2['ground_status'].astype('int64')

In [166]:
#使う特徴量の設定
id = {"race_id",
      "horse_id"
    }

feature = [
           "race_id",
           "horse_id",
           "rider_id",
           "horse_number",
           "age",
           "sex",
#             "odds",
#             'popular',
            'horse_weight',
            'ground_status',
            'tame_id',
            'inbreeding_1',
            'inbreeding_2'
            ]

target = ['rank']

X_id = df_main_2[feature]
X_id['horse_weight'] = X_id['horse_weight'].astype('int64')
X_id['tame_id'] = X_id['tame_id'].astype('int64')
X_id['inbreeding_1'] = X_id['inbreeding_1'].astype('int64')
y = df_main_2[target]

C:\Users\kawam\AppData\Local\Temp\ipykernel_2276\1194713136.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_id['horse_weight'] = X_id['horse_weight'].astype('int64')


In [143]:
X_id

,race_id,horse_id,rider_id,horse_number,age,sex,horse_weight,ground_status
0,202009020411,2016104750,1102,12,4,1,454,1
1,202109020411,2017105335,1088,8,4,1,422,4
2,202209020411,2018105554,1014,11,4,1,498,1
3,201709020411,2013105788,5339,14,4,0,502,1
4,201709020411,2013106099,1018,1,4,0,480,1
...,...,...,...,...,...,...,...,...
80,202209020411,2015101654,1124,15,7,2,468,1
81,201809020411,2010102459,700,6,8,1,474,1
82,201709020411,2009102678,700,9,8,0,500,1
83,201809020411,2010103602,732,10,8,0,510,1


In [129]:
#使う特徴量の設定
id = {"race_id",
      "horse_id"
    }

feature = [
           "race_id",
           "horse_id",
           "rider_id",
           "horse_number",
           "age",
           "sex",
#             "odds",
#             'popular',
            'horse_weight',
            'ground_status',
#             'tame_id',
#             'inbreeding_1',
#             'inbreeding_2'
            ]

target = ['rank']

today_race_X = df_target_2[feature]
today_race_X['horse_weight'] = today_race_X['horse_weight'].astype('int64')
# today_race_X['tame_id'] = today_race_X['tame_id'].astype('int64')
# today_race_X['inbreeding_1'] = today_race_X['inbreeding_1'].astype('int64')
today_race_X.to_csv("../data/main/today.csv",index=False)

C:\Users\kawam\AppData\Local\Temp\ipykernel_2276\1324036470.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  today_race_X['horse_weight'] = today_race_X['horse_weight'].astype('int64')


In [130]:
today_race_X_with_name = pd.merge(today_race_X, targethorse_df[['horse_id', 'horse_name']], on='horse_id', how='left')
today_race_X.to_csv("../data/main/today_withname.csv",index=False)

In [153]:
pred_df = pd.DataFrame()
#スコア結果を入れるリスト
reports = []
auc_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
support_scores = []

#予測結果を入れるリスト
pred_df_list = []

def objective(trial):
    
# Optuna
    # ハイパーパラメータの探索範囲
    params = {
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        'verbose': -1
    }
    
    #LGBMRankerの設定
    ranker = LGBMRanker(**params)

# K-fold
    #kflodの設定
    kfold = KFold(n_splits=2, shuffle=True, random_state=42)

    #kflodで学習用と検証用に分ける
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_id, y)):
        #id入りの学習データ
        X_train_id, X_val_id = X_id.iloc[train_idx], X_id.iloc[val_idx]
        
        #id抜きの学習データ
        X = X_id.drop(id, axis =1)
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        
        #検証データ
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

# Fit
        # train_basketsを計算
        train_baskets = X_train_id.groupby(["race_id"])["horse_id"].count().values

        # モデルの学習
        model = ranker.fit(X_train,
                           y_train,
                           group=train_baskets)

# Predict
        # モデルの予測
        y_pred = model.predict(X_val)

        #結果を入れるデータフレームを用意
        pred_df_fold = pd.DataFrame({
           "race_id": X_val_id['race_id'],
           "horse_id": X_val_id['horse_id'],
           "rank":y_val['rank'],
           "pred": y_pred
           })

        #そのデータフレームをリストに保存
        pred_df_list.append(pred_df_fold)

# Rank
    # 各foldの予測結果を結合してpred_dfを作成
    pred_df = pd.concat(pred_df_list, axis=0).reset_index(drop=True)

    #予測した結果をrace_idごとでグループ化し順位をつける
    pred_df['pred_rank'] = pred_df.groupby('race_id')['pred'].rank(method='min', ascending=False)

    #3位以内ならprizeを1にする
    pred_df['rank_prize'] = pred_df['rank'].apply(lambda x: 1 if x <= 3 else 0)
    pred_df['pred_rank_prize'] = pred_df['pred_rank'].apply(lambda x: 1 if x <= 3 else 0)
    print(pred_df)

#Score
    #正確度を計算
    auc_score = metrics.accuracy_score(pred_df['rank_prize'], pred_df['pred_rank_prize'])
        
    #auc_scoresのリストに保存
    auc_scores.append(auc_score)
    
    #f1を計算            
    f1 = f1_score(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    
    #reportを計算
    report = metrics.classification_report(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    report_num = metrics.precision_recall_fscore_support(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    reports.append(report)

    #reportから情報をとる
    precision_scores.append(report_num[0])
    recall_scores.append(report_num[1])
    f1_scores.append(report_num[2])
    support_scores.append(report_num[3])
    
    return f1

In [154]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2023-04-03 12:00:41,336] A new study created in memory with name: no-name-c0de3fe3-1417-489c-9e83-11f27da32699
[I 2023-04-03 12:00:41,447] Trial 0 finished with value: 0.16216216216216214 and parameters: {'num_leaves': 53, 'learning_rate': 0.09497032914123206}. Best is trial 0 with value: 0.16216216216216214.
[I 2023-04-03 12:00:41,543] Trial 1 finished with value: 0.14285714285714285 and parameters: {'num_leaves': 32, 'learning_rate': 0.07351144724287301}. Best is trial 0 with value: 0.16216216216216214.


         race_id    horse_id  rank      pred  pred_rank  rank_prize  \
0   202009020411  2016104750     2  0.112237        2.0           1   
1   201709020411  2013106099     7 -0.874089       11.0           0   
2   201709020411  2013102301    14 -0.141523        5.0           0   
3   201809020411  2014105258     2 -0.904663       13.0           1   
4   201809020411  2014106160     5 -0.874089       11.0           0   
..           ...         ...   ...       ...        ...         ...   
80  202009020411  2013105788    11 -0.059275        3.0           0   
81  202109020411  2014104386    11 -1.194719       11.0           0   
82  202209020411  2015101654     7  1.157912        1.0           0   
83  201809020411  2010102459     9 -1.413972       16.0           0   
84  201809020411  2010103602    15 -0.419453       10.0           0   

    pred_rank_prize  
0                 1  
1                 0  
2                 0  
3                 0  
4                 0  
..             

[I 2023-04-03 12:00:41,649] Trial 2 finished with value: 0.10810810810810811 and parameters: {'num_leaves': 60, 'learning_rate': 0.02392684598264929}. Best is trial 0 with value: 0.16216216216216214.
[I 2023-04-03 12:00:41,754] Trial 3 finished with value: 0.06593406593406594 and parameters: {'num_leaves': 32, 'learning_rate': 0.08741041129755392}. Best is trial 0 with value: 0.16216216216216214.
[I 2023-04-03 12:00:41,857] Trial 4 finished with value: 0.05504587155963303 and parameters: {'num_leaves': 41, 'learning_rate': 0.04805634093535318}. Best is trial 0 with value: 0.16216216216216214.


          race_id    horse_id  rank      pred  pred_rank  rank_prize  \
0    202009020411  2016104750     2  0.112237        4.0           1   
1    201709020411  2013106099     7 -0.874089       49.0           0   
2    201709020411  2013102301    14 -0.141523       37.0           0   
3    201809020411  2014105258     2 -0.904663       55.0           1   
4    201809020411  2014106160     5 -0.874089       53.0           0   
..            ...         ...   ...       ...        ...         ...   
335  202009020411  2013105788    11 -0.074477       26.0           0   
336  202109020411  2014104386    11 -1.208421       47.0           0   
337  202209020411  2015101654     7  1.191519        1.0           0   
338  201809020411  2010102459     9 -1.451652       64.0           0   
339  201809020411  2010103602    15 -0.157999       45.0           0   

     pred_rank_prize  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0

[I 2023-04-03 12:00:41,969] Trial 5 finished with value: 0.047244094488188976 and parameters: {'num_leaves': 40, 'learning_rate': 0.07415922594624004}. Best is trial 0 with value: 0.16216216216216214.
[I 2023-04-03 12:00:42,072] Trial 6 finished with value: 0.041379310344827586 and parameters: {'num_leaves': 46, 'learning_rate': 0.02361743250335595}. Best is trial 0 with value: 0.16216216216216214.
[I 2023-04-03 12:00:42,176] Trial 7 finished with value: 0.03680981595092024 and parameters: {'num_leaves': 75, 'learning_rate': 0.08907866783321058}. Best is trial 0 with value: 0.16216216216216214.


          race_id    horse_id  rank      pred  pred_rank  rank_prize  \
0    202009020411  2016104750     2  0.112237        5.0           1   
1    201709020411  2013106099     7 -0.874089       89.0           0   
2    201709020411  2013102301    14 -0.141523       76.0           0   
3    201809020411  2014105258     2 -0.904663      100.0           1   
4    201809020411  2014106160     5 -0.874089       98.0           0   
..            ...         ...   ...       ...        ...         ...   
590  202009020411  2013105788    11  0.005514       18.0           0   
591  202109020411  2014104386    11 -0.036274       59.0           0   
592  202209020411  2015101654     7  0.032224       31.0           0   
593  201809020411  2010102459     9 -0.057793       80.0           0   
594  201809020411  2010103602    15  0.011954       32.0           0   

     pred_rank_prize  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0

[I 2023-04-03 12:00:42,278] Trial 8 finished with value: 0.03314917127071823 and parameters: {'num_leaves': 62, 'learning_rate': 0.03984720949325772}. Best is trial 0 with value: 0.16216216216216214.
[I 2023-04-03 12:00:42,381] Trial 9 finished with value: 0.030150753768844223 and parameters: {'num_leaves': 18, 'learning_rate': 0.00602537569802948}. Best is trial 0 with value: 0.16216216216216214.


          race_id    horse_id  rank      pred  pred_rank  rank_prize  \
0    202009020411  2016104750     2  0.112237        7.0           1   
1    201709020411  2013106099     7 -0.874089      129.0           0   
2    201709020411  2013102301    14 -0.141523      109.0           0   
3    201809020411  2014105258     2 -0.904663      145.0           1   
4    201809020411  2014106160     5 -0.874089      143.0           0   
..            ...         ...   ...       ...        ...         ...   
845  202009020411  2013105788    11  0.000609       33.0           0   
846  202109020411  2014104386    11 -0.164611      101.0           0   
847  202209020411  2015101654     7  0.149530       18.0           0   
848  201809020411  2010102459     9 -0.197135      133.0           0   
849  201809020411  2010103602    15  0.012430       43.0           0   

     pred_rank_prize  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0

In [155]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_num_leaves,state
0,0,0.162162,2023-04-03 12:00:41.338687,2023-04-03 12:00:41.447643,0 days 00:00:00.108956,0.094970,53,COMPLETE
1,1,0.142857,2023-04-03 12:00:41.449655,2023-04-03 12:00:41.543942,0 days 00:00:00.094287,0.073511,32,COMPLETE
2,2,0.108108,2023-04-03 12:00:41.545946,2023-04-03 12:00:41.649986,0 days 00:00:00.104040,0.023927,60,COMPLETE
3,3,0.065934,2023-04-03 12:00:41.650987,2023-04-03 12:00:41.754785,0 days 00:00:00.103798,0.087410,32,COMPLETE
4,4,0.055046,2023-04-03 12:00:41.757810,2023-04-03 12:00:41.857359,0 days 00:00:00.099549,0.048056,41,COMPLETE
5,5,0.047244,2023-04-03 12:00:41.859361,2023-04-03 12:00:41.968182,0 days 00:00:00.108821,0.074159,40,COMPLETE
6,6,0.041379,2023-04-03 12:00:41.971206,2023-04-03 12:00:42.072491,0 days 00:00:00.101285,0.023617,46,COMPLETE
7,7,0.036810,2023-04-03 12:00:42.073492,2023-04-03 12:00:42.176354,0 days 00:00:00.102862,0.089079,75,COMPLETE
8,8,0.033149,2023-04-03 12:00:42.178366,2023-04-03 12:00:42.278668,0 days 00:00:00.100302,0.039847,62,COMPLETE
9,9,0.030151,2023-04-03 12:00:42.280682,2023-04-03 12:00:42.381642,0 days 00:00:00.100960,0.006025,18,COMPLETE


In [156]:
#Optunaによる、ベストパラメータの取得
best_params = study.best_params

In [159]:
best_ranker = LGBMRanker(**best_params)

In [160]:
train_baskets = X_id.groupby(["race_id"])["horse_id"].count().values
X = X_id.drop(id, axis =1)
model = best_ranker.fit(X,
                        y,
                        group=train_baskets)

In [161]:
result = model.predict(X)

In [162]:
result

array([-0.24611401,  0.17183573, -0.93647092, -0.86905506, -1.63254765,
        1.21864993, -0.86905506, -1.63254765,  0.32393374, -1.46569935,
       -0.2370471 , -0.72502161, -0.49448702, -1.26015242, -0.09506179,
       -1.02323256,  0.16090777,  0.94340523, -0.13800532, -1.13224975,
       -1.46569935, -0.28904063, -0.9916218 , -0.65016028, -1.24706237,
       -1.53333645, -1.35272854, -0.5049903 , -1.58157989, -0.62874024,
       -0.71198033, -1.74048231,  1.05051875, -1.74048231, -1.41166668,
       -1.22873937,  0.49942713, -0.50048309, -0.91702249, -1.36646279,
       -1.58157989,  0.11695616, -1.36430973, -1.49416216, -0.41087891,
       -0.41981409, -1.40535034, -0.87653095, -0.58353636,  0.90779833,
       -1.13039477, -1.14090205, -1.60177191,  0.28971883, -0.6262319 ,
       -1.70098311, -1.13039477, -0.67575765,  0.34376394, -0.66089493,
       -0.93961699, -0.53450511, -1.205032  , -0.64351478,  0.44481653,
        1.38812855, -1.22039706, -0.42078234,  0.30222955, -0.11

In [163]:
with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [164]:
#predict
train_baskets = X_id.groupby(["race_id"])["horse_id"].count().values
X = X_id.drop(["race_id", "horse_id"], axis=1)
y_pred = model.predict(X, group=train_baskets)

# 結果を入れるデータフレームを用意
pred_df = pd.DataFrame({
   "race_id": X_id['race_id'],
   "horse_id": X_id['horse_id'],
   "rank": y['rank'],
   "pred": y_pred
})

#予測した結果をrace_idごとでグループ化し順位をつける
pred_df['pred_rank'] = pred_df.groupby('race_id')['pred'].rank(method='min', ascending=False)

#3位以内ならprizeを1にする
pred_df['rank_prize'] = pred_df['rank'].apply(lambda x: 1 if x <= 3 else 0)
pred_df['pred_rank_prize'] = pred_df['pred_rank'].apply(lambda x: 1 if x <= 3 else 0)

In [165]:
pred_df

,race_id,horse_id,rank,pred,pred_rank,rank_prize,pred_rank_prize
0,202009020411,2016104750,2,-0.246114,3.0,1,1
1,202109020411,2017105335,1,0.171836,2.0,1,1
2,202209020411,2018105554,8,-0.936471,11.0,0,0
3,201709020411,2013105788,4,-0.869055,6.0,0,0
4,201709020411,2013106099,7,-1.632548,13.0,0,0
...,...,...,...,...,...,...,...
80,202209020411,2015101654,7,0.482948,2.0,0,1
81,201809020411,2010102459,9,-0.557767,8.0,0,0
82,201709020411,2009102678,10,-0.875681,7.0,0,0
83,201809020411,2010103602,15,-0.875681,12.0,0,0


In [47]:
#predict
train_baskets = today_race_X.groupby(["race_id"])["horse_id"].count().values
X = today_race_X.drop(["race_id", "horse_id"], axis=1)
y_pred = model.predict(X, group=train_baskets)

# 結果を入れるデータフレームを用意
pred_df = pd.DataFrame({
   "race_id": today_race_X['race_id'],
   "horse_id": today_race_X['horse_id'],
    "horse_name":today_race_X_with_name['horse_name'],
        "horse_number":today_race_X['horse_number'],
   "pred": y_pred
})

#予測した結果をrace_idごとでグループ化し順位をつける
pred_df['pred_rank'] = pred_df.groupby('race_id')['pred'].rank(method='min', ascending=False)

#3位以内ならprizeを1にする
pred_df['pred_rank_prize'] = pred_df['pred_rank'].apply(lambda x: 1 if x <= 3 else 0)

In [48]:
pred_df.sort_values('horse_number')

,race_id,horse_id,horse_name,horse_number,pred,pred_rank,pred_rank_prize
0,202309020411,2018105081,ジェラルディーナ,1,0.0,1.0,1
1,202309020411,2018100927,マリアエレーナ,2,0.0,1.0,1
3,202309020411,2016100915,モズベッロ,3,0.0,1.0,1
4,202309020411,2018102348,ノースブリッジ,4,0.0,1.0,1
5,202309020411,2018105074,ワンダフルタウン,5,0.0,1.0,1
6,202309020411,2017105567,ェルトライゼンデ,6,0.0,1.0,1
7,202309020411,2019102879,マテンロウレオ,7,0.0,1.0,1
8,202309020411,2019101782,ラーグルフ,8,0.0,1.0,1
9,202309020411,2018100274,ジャックドール,9,0.0,1.0,1
10,202309020411,2017105376,ポタジェ,10,0.0,1.0,1


In [49]:
# Unpickle model
model = pd.read_pickle('./model.pickle')

In [50]:
# one = float(request.POST.get('one'))
# two = float(request.POST.get('two'))
# three = float(request.POST.get('three'))
# four = float(request.POST.get('four'))
# five = float(request.POST.get('five'))
# six = float(request.POST.get('six'))
# seven = float(request.POST.get('seven'))
# eight = float(request.POST.get('eight'))
# nine = float(request.POST.get('nine'))
# ten = float(request.POST.get('ten'))
# eleven = float(request.POST.get('eleven'))
# twelve = float(request.POST.get('twelve'))
# thirteen = float(request.POST.get('thirteen'))
# fourteen = float(request.POST.get('fourteen'))
# fifteen = float(request.POST.get('fifteen'))
# sixteen = float(request.POST.get('sixteen'))
# seventeen = float(request.POST.get('seventeen'))
# eighteen = float(request.POST.get('eighteen'))

In [51]:
one = 500
two = 500
three = 500
four = 500
five = 500
six = 500
seven = 500
eight = 500
nine = 500
ten = 500
eleven = 500
twelve = 500
thirteen = 500
fourteen = 500
fifteen = 500
sixteen = 500
# seventeen = 500
# eighteen = 500

In [52]:
today_race_X = pd.read_csv('../data/main/today.csv')
today_race_X_withname = pd.read_csv('../data/main/today_withname.csv')

In [53]:
today_race_X.loc[today_race_X['horse_number'] == 1, 'horse_weight'] = one
today_race_X.loc[today_race_X['horse_number'] == 2, 'horse_weight'] = two
today_race_X.loc[today_race_X['horse_number'] == 3, 'horse_weight'] = three
today_race_X.loc[today_race_X['horse_number'] == 4, 'horse_weight'] = four
today_race_X.loc[today_race_X['horse_number'] == 5, 'horse_weight'] = five
today_race_X.loc[today_race_X['horse_number'] == 6, 'horse_weight'] = six
today_race_X.loc[today_race_X['horse_number'] == 7, 'horse_weight'] = seven
today_race_X.loc[today_race_X['horse_number'] == 8, 'horse_weight'] = eight
today_race_X.loc[today_race_X['horse_number'] == 9, 'horse_weight'] = nine
today_race_X.loc[today_race_X['horse_number'] == 10, 'horse_weight'] = ten
today_race_X.loc[today_race_X['horse_number'] == 11, 'horse_weight'] = eleven
today_race_X.loc[today_race_X['horse_number'] == 12, 'horse_weight'] = twelve
today_race_X.loc[today_race_X['horse_number'] == 13, 'horse_weight'] = thirteen
today_race_X.loc[today_race_X['horse_number'] == 14, 'horse_weight'] = fourteen
today_race_X.loc[today_race_X['horse_number'] == 15, 'horse_weight'] = fifteen
today_race_X.loc[today_race_X['horse_number'] == 16, 'horse_weight'] = sixteen
# today_race_X.loc[today_race_X['horse_number'] == 17, 'horse_weight'] = seventeen
# today_race_X.loc[today_race_X['horse_number'] == 18, 'horse_weight'] = eighteen

In [54]:
today_race_X = today_race_X.sort_values('horse_number')
today_race_X

,race_id,horse_id,rider_id,horse_number,age,sex,horse_weight,ground_status
0,202309020411,2018105081,1174,1,5,1,500,1
1,202309020411,2018100927,1115,2,5,1,500,1
3,202309020411,2016100915,1171,3,7,0,500,1
4,202309020411,2018102348,5203,4,5,0,500,1
5,202309020411,2018105074,1018,5,5,0,500,1
6,202309020411,2017105567,1088,6,6,0,500,1
7,202309020411,2019102879,660,7,4,0,500,1
8,202309020411,2019101782,5386,8,4,0,500,1
9,202309020411,2018100274,666,9,5,0,500,1
10,202309020411,2017105376,1163,10,6,0,500,1


In [55]:
#predict
train_baskets = today_race_X.groupby(["race_id"])["horse_id"].count().values
X = today_race_X.drop(["race_id", "horse_id"], axis=1)
y_pred = model.predict(X, group=train_baskets)

In [56]:
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [57]:
rank = np.arange(1, 19)
sorted_rank = rank[np.argsort(y_pred)[::-1]]

In [58]:
sorted_rank[0]

16